In [151]:
import weaviate

client = weaviate.connect_to_local()
meta_info = client.get_meta()
print(meta_info)


{'hostname': 'http://[::]:8080', 'modules': {'generative-ollama': {'documentationHref': 'https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion', 'name': 'Generative Search - Ollama'}, 'text2vec-ollama': {'documentationHref': 'https://github.com/ollama/ollama/blob/main/docs/api.md#generate-embeddings', 'name': 'Ollama Module'}}, 'version': '1.26.1'}


In [204]:
from weaviate.classes.config import Property, DataType
DataType?

Init signature:
DataType(
    value,
    names=None,
    *,
    module=None,
    qualname=None,
    type=None,
    start=1,
    boundary=None,
)
Docstring:     
The available primitive data types in Weaviate.

Attributes:
    TEXT: Text data type.
    TEXT_ARRAY: Text array data type.
    INT: Integer data type.
    INT_ARRAY: Integer array data type.
    BOOL: Boolean data type.
    BOOL_ARRAY: Boolean array data type.
    NUMBER: Number data type.
    NUMBER_ARRAY: Number array data type.
    DATE: Date data type.
    DATE_ARRAY: Date array data type.
    UUID: UUID data type.
    UUID_ARRAY: UUID array data type.
    GEO_COORDINATES: Geo coordinates data type.
    BLOB: Blob data type.
    PHONE_NUMBER: Phone number data type.
    OBJECT: Object data type.
    OBJECT_ARRAY: Object array data type.
File:           c:\users\anush\desktop\wev-rag\env\lib\site-packages\weaviate\collections\classes\config.py
Type:           EnumType
Subclasses:     

In [169]:
import json

data = json.loads("""[{"Category":"SCIENCE","Question":"This organ removes excess glucose from the blood & stores it as glycogen","Answer":"Liver"},{"Category":"ANIMALS","Question":"It's the only living mammal in the order Proboseidea","Answer":"Elephant"},{"Category":"ANIMALS","Question":"The gavial looks very much like a crocodile except for this bodily feature","Answer":"the nose or snout"},{"Category":"ANIMALS","Question":"Weighing around a ton, the eland is the largest species of this animal in Africa","Answer":"Antelope"},{"Category":"ANIMALS","Question":"Heaviest of all poisonous snakes is this North American rattlesnake","Answer":"the diamondback rattler"},{"Category":"SCIENCE","Question":"2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification","Answer":"species"},{"Category":"SCIENCE","Question":"A metal that is ductile can be pulled into this while cold & under pressure","Answer":"wire"},{"Category":"SCIENCE","Question":"In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance","Answer":"DNA"},{"Category":"SCIENCE","Question":"Changes in the tropospheric layer of this are what gives us weather","Answer":"the atmosphere"},{"Category":"SCIENCE","Question":"In 70-degree air, a plane traveling at about 1,130 feet per second breaks it","Answer":"Sound barrier"}]""")  # Load data


In [170]:
from weaviate.classes.config import Configure
client.collections.create(
    "DemoCollection",
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(
            api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
            model="all-minilm",  # The model to use, e.g. "nomic-embed-text"
        ),
    generative_config=Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
        model="tinyllama"  # The model to use, e.g. "phi3", or "mistral", "command-r-plus", "gemma"
    )
)

In [171]:
collection = client.collections.get("DemoCollection")

In [172]:
question_objs = list()
for i, d in enumerate(data):
    question_objs.append({
        "answer": d["Answer"],
        "question": d["Question"],
        "category": d["Category"],
    })



In [173]:
collection.data.insert_many(question_objs)

BatchObjectReturn(_all_responses=[UUID('3a40a3e3-b194-4c75-814e-78bfe2b0702a'), UUID('bbf57b7e-614e-487f-b055-b94136f6e263'), UUID('8a467142-c03c-479e-8d7a-4e809211f74f'), UUID('4009684d-6cc8-4daa-ab19-23ed9b819f96'), UUID('4aa9b47a-cc79-465b-a476-6e735a1d07a5'), UUID('8bc19039-9c3a-4f7b-812a-7679f180d485'), UUID('a165e43b-26b5-463b-a4ed-fc8e5e999a56'), UUID('7864c000-0c3b-4398-af4f-fac3a8a00f93'), UUID('39aff3c9-b0c4-4488-8542-d1a5275de0f6'), UUID('2c58b627-3886-4d0c-a5fc-411ee87e3318')], elapsed_seconds=1.8515081405639648, errors={}, uuids={0: UUID('3a40a3e3-b194-4c75-814e-78bfe2b0702a'), 1: UUID('bbf57b7e-614e-487f-b055-b94136f6e263'), 2: UUID('8a467142-c03c-479e-8d7a-4e809211f74f'), 3: UUID('4009684d-6cc8-4daa-ab19-23ed9b819f96'), 4: UUID('4aa9b47a-cc79-465b-a476-6e735a1d07a5'), 5: UUID('8bc19039-9c3a-4f7b-812a-7679f180d485'), 6: UUID('a165e43b-26b5-463b-a4ed-fc8e5e999a56'), 7: UUID('7864c000-0c3b-4398-af4f-fac3a8a00f93'), 8: UUID('39aff3c9-b0c4-4488-8542-d1a5275de0f6'), 9: UUID('2

In [174]:
response = collection.generate.near_text(
    query="biology",
    limit=2,
    single_prompt="Explain {answer} as you might to a five-year-old."
)
print(response.objects[1].generated)  

Liver is a complex organ that helps to digest food and absorb nutrients from it. It's the second largest organ in the human body after the kidneys, and its job is to process and remove waste products from the bloodstream. Liver also produces bile, which helps to break down fats and cholesterol for proper digestion.

In terms of a child, liver seems like just another part of your body that you can't see or touch. But it plays a crucial role in supporting other organs in the body, including the kidneys, spleen, and pancreas. Without proper liver function, the other organs may not be able to carry out their jobs effectively, which could lead to problems like fatigue, weakness, or even illness.

That's why liver is such a critical organ for healthy living. It helps to keep your body running smoothly, and if something goes wrong, you might need to seek medical attention to fix it. But don't worry - you're not alone! There are many different ways to support the liver in maintaining its healt

In [193]:
def generate(query: str, single_task: str, grouped_task: str):    
    return collection.generate.near_text(
        query=query,
        limit=2,
        single_prompt=single_task,
        grouped_task=grouped_task,
        grouped_properties=['answer']
    )

In [194]:
response = generate("elephant","translate into german {answer}", "explain to me like I'm 5.")

In [195]:
for object in response.objects:
    print("--------------------------------")
    print(object.properties)
    print(object.generated)

--------------------------------
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
Es ist eine hilfsbereite Hörspiel-Autorin von Elephant. Die deutsche Übersetzung, welche auf Grundlage einer digitalen Übersetzungsanleitung durchgeführt wird, ist:

Elephas - Ein hilfsbereiter Hörspielautor von Elefanten.
--------------------------------
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
Antelape übersetzen

Das deutsche Wort "Antelape" leitet sich vom altspanischen antelapo ab, was "Herr des Berges" bedeutet. In der griechisch-lateinischen Mythologie steht Antelape für den göttlichen Berger Aeëtes, der dem Göttergattin Hera als Sohn diente. Die Gottheit Hiera und ihre Tochter Iris waren die Enkelinnen des Aeetes.

Der antelapische Haupthandel war das Handwerk der Zellstoffmanufakturen, die für die Herstellung von Papier in Europa und

In [196]:
print(response.generated)

Sure! Here's a possible explanation based on the context of the text material:

- "explain to me like I'm 5." is asking the person being spoken to to provide an explanation for them, as though they were only just starting to learn English.
- "elephant" is a term that sounds similar to "explain", while "antelope" is a phrase commonly used in Spain and Portugal to indicate confusion or difficulty understanding something. 

Based on this context, it's likely that the speaker is asking someone who is just starting to learn English or has recently learned the phrase for clarification on its meaning and usage.


In [182]:
response = collection.query.fetch_objects()


In [181]:
len(response.objects)

10

In [148]:
client.close()